# **OBSERVATION**

After the EDA of all the datasets provided to us we can observe that:
- The dataset is imbalanced.
- Many features contain Nan or XNA value. For Nan values we would have to take mean of the feature or use a simple model like KNN to fill the missing section. Features with too many missing values could be removed decreasing the dimensionality of our model.
- There are some wrong entries in the features which need to be removed.
- There are also some values which are outdated and should be removed as it wont be contributing in the model in anyway
- There are also some features which have same type of inputs in it and are just increasing the dimensionality and could be removed too. 
- There would be some features which might not contribute too much for the model and could be removed too with the help of coorelation with the target.
- Not all sorts of outliers could be removed as some of them may really be adding some value to the model. 
- There are some features which provide really good insight in discriminating the defaulters from the non defaulters

Keeping all this in mind we could proceed with the preprocessing of the data

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import os
import matplotlib.pyplot as plt
import seaborn as sns

# **PREPROCESSING**

The adjustments we apply to our data before feeding it to the algorithm are referred to as pre-processing. Data preprocessing is a method for converting unclean data into a clean data set. In other words, anytime data is received from various sources, it is collected in raw format, which makes analysis impossible.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.6.0


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':'1R4_biFY4l4jwmaMMzOh5cgB1nB6Eb1z1'}) # replace the id with id of file you want to access
downloaded.GetContentFile('application_train.csv') 


In [ ]:
downloaded = drive.CreateFile({'id':'1ni9rz70QVjmB026wuY2qgjbgw-MV3EAr'}) # replace the id with id of file you want to access
downloaded.GetContentFile('application_test.csv') 


# **TRAIN.CSV AND TEST.CSV**

We have already seen some abnormalities in some of the features while doing the EDA so we shall solve all of those first

In [ ]:
train = pd.read_csv('application_train.csv')
print(train.shape)
test=pd.read_csv("application_test.csv")
print(test.shape)

(307511, 122)
(48744, 121)


In [ ]:
print(train.CODE_GENDER.value_counts())
index = train[ train['CODE_GENDER'] == "XNA" ].index
train.drop(index , inplace=True)


F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64


In [ ]:
test.CODE_GENDER.value_counts()

F    32678
M    16066
Name: CODE_GENDER, dtype: int64

Dropped the XNA rows for code gender in train as there were only 4 in total

In [ ]:
train.DAYS_EMPLOYED.quantile([.0, .1, .25, .5, .75, 1])
train['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

0.00    -17912.0
0.10     -4881.0
0.25     -2760.0
0.50     -1213.0
0.75      -289.0
1.00    365243.0
Name: DAYS_EMPLOYED, dtype: float64

In [ ]:
test.DAYS_EMPLOYED.quantile([.0, .1, .25, .5, .75, 1])

0.00    -17463.0
0.10     -4929.8
0.25     -2910.0
0.50     -1293.0
0.75      -296.0
1.00    365243.0
Name: DAYS_EMPLOYED, dtype: float64

In [ ]:
test['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

In [ ]:
print("train")
print(train.DAYS_EMPLOYED.quantile([.0, .1, .25, .5, .75, 1]))
print("="*50)
print("test")
print(test.DAYS_EMPLOYED.quantile([.0, .1, .25, .5, .75, 1]))

train
0.00    -17912.0
0.10     -4881.0
0.25     -2760.0
0.50     -1213.0
0.75      -289.0
1.00    365243.0
Name: DAYS_EMPLOYED, dtype: float64
test
0.00   -17463.00
0.10    -5410.00
0.25    -3328.75
0.50    -1765.00
0.75     -861.00
1.00       -1.00
Name: DAYS_EMPLOYED, dtype: float64


Replaced the number of days 365243 with np.nan as its practically impossible for someone to be employed for this long

In [ ]:
train=train.drop("FLAG_DOCUMENT_2",1)
test=test.drop("FLAG_DOCUMENT_2",1)

train=train.drop("FLAG_DOCUMENT_4",1)
test=test.drop("FLAG_DOCUMENT_4",1)

train=train.drop("FLAG_DOCUMENT_7",1)
test=test.drop("FLAG_DOCUMENT_7",1)

train=train.drop("FLAG_DOCUMENT_10",1)
test=test.drop("FLAG_DOCUMENT_10",1)

train=train.drop("FLAG_DOCUMENT_10",1)
test=test.drop("FLAG_DOCUMENT_10",1)

train=train.drop("FLAG_DOCUMENT_12",1)
test=test.drop("FLAG_DOCUMENT_12",1)

train=train.drop("FLAG_DOCUMENT_17",1)
test=test.drop("FLAG_DOCUMENT_17",1)

train=train.drop("FLAG_DOCUMENT_19",1)
test=test.drop("FLAG_DOCUMENT_19",1)

train=train.drop("FLAG_DOCUMENT_20",1)
test=test.drop("FLAG_DOCUMENT_20",1)

0    307494
1        13
Name: FLAG_DOCUMENT_2, dtype: int64


In [ ]:

print(train.FLAG_DOCUMENT_4.value_counts())
train=train.drop("FLAG_DOCUMENT_4",1)
test=test.drop("FLAG_DOCUMENT_4",1)

0    307482
1        25
Name: FLAG_DOCUMENT_4, dtype: int64


In [ ]:
print(train[train["FLAG_DOCUMENT_7"]==1].TARGET.value_counts())
print(train.FLAG_DOCUMENT_7.value_counts())
train=train.drop("FLAG_DOCUMENT_7",1)
test=test.drop("FLAG_DOCUMENT_7",1)

0    56
1     3
Name: TARGET, dtype: int64
0    307448
1        59
Name: FLAG_DOCUMENT_7, dtype: int64


In [ ]:

print(train.FLAG_DOCUMENT_10.value_counts())
train=train.drop("FLAG_DOCUMENT_10",1)
test=test.drop("FLAG_DOCUMENT_10",1)

0    307500
1         7
Name: FLAG_DOCUMENT_10, dtype: int64


In [ ]:

print(train.FLAG_DOCUMENT_10.value_counts())
train=train.drop("FLAG_DOCUMENT_10",1)
test=test.drop("FLAG_DOCUMENT_10",1)
print(train.FLAG_DOCUMENT_12.value_counts())
train=train.drop("FLAG_DOCUMENT_12",1)
test=test.drop("FLAG_DOCUMENT_12",1)

0    307505
1         2
Name: FLAG_DOCUMENT_12, dtype: int64


In [ ]:

print(train.FLAG_DOCUMENT_10.value_counts())
train=train.drop("FLAG_DOCUMENT_10",1)
test=test.drop("FLAG_DOCUMENT_10",1)
print(train.FLAG_DOCUMENT_12.value_counts())
train=train.drop("FLAG_DOCUMENT_12",1)
test=test.drop("FLAG_DOCUMENT_12",1)
print(train.FLAG_DOCUMENT_17.value_counts())
train=train.drop("FLAG_DOCUMENT_17",1)
test=test.drop("FLAG_DOCUMENT_17",1)

0    307425
1        82
Name: FLAG_DOCUMENT_17, dtype: int64


In [ ]:

print(train.FLAG_DOCUMENT_10.value_counts())
train=train.drop("FLAG_DOCUMENT_10",1)
test=test.drop("FLAG_DOCUMENT_10",1)
print(train.FLAG_DOCUMENT_12.value_counts())
train=train.drop("FLAG_DOCUMENT_12",1)
test=test.drop("FLAG_DOCUMENT_12",1)
print(train.FLAG_DOCUMENT_17.value_counts())
train=train.drop("FLAG_DOCUMENT_17",1)
test=test.drop("FLAG_DOCUMENT_17",1)
print(train.FLAG_DOCUMENT_19.value_counts())
train=train.drop("FLAG_DOCUMENT_19",1)
test=test.drop("FLAG_DOCUMENT_19",1)

0    307324
1       183
Name: FLAG_DOCUMENT_19, dtype: int64


In [ ]:

print(train.FLAG_DOCUMENT_10.value_counts())
train=train.drop("FLAG_DOCUMENT_10",1)
test=test.drop("FLAG_DOCUMENT_10",1)
print(train.FLAG_DOCUMENT_12.value_counts())
train=train.drop("FLAG_DOCUMENT_12",1)
test=test.drop("FLAG_DOCUMENT_12",1)
print(train.FLAG_DOCUMENT_17.value_counts())
train=train.drop("FLAG_DOCUMENT_17",1)
test=test.drop("FLAG_DOCUMENT_17",1)
print(train.FLAG_DOCUMENT_19.value_counts())
train=train.drop("FLAG_DOCUMENT_19",1)
test=test.drop("FLAG_DOCUMENT_19",1)
print(train.FLAG_DOCUMENT_20.value_counts())
train=train.drop("FLAG_DOCUMENT_20",1)
test=test.drop("FLAG_DOCUMENT_20",1)

0    307351
1       156
Name: FLAG_DOCUMENT_20, dtype: int64


In [ ]:
print(train.shape)
print(test.shape)

(307507, 114)
(48744, 113)


FEATURE ENGINEERING

REFERENCE:
https://www.gopaysense.com/blog/important-factors-personal-loan-applications/
https://www.businesstoday.in/latest/economy-politics/story/10-factors-banks-consider-before-approving-your-home-loan-78784-2017-05-08

Banks while lending loans will look into multiple things of the client such as their credit ratio, annual income, total work experience and so on before lending loans. These things play an important role and thus we shall add these in our dataset too as it might help getting better results

In [ ]:
train['annuity_income_percentage'] = train['AMT_ANNUITY'] / train['AMT_INCOME_TOTAL']
train['car_to_birth_ratio'] = train['OWN_CAR_AGE'] / train['DAYS_BIRTH']
train['car_to_employ_ratio'] = train['OWN_CAR_AGE'] / train['DAYS_EMPLOYED']
train['children_ratio'] = train['CNT_CHILDREN'] / train['CNT_FAM_MEMBERS']
train['Annual_PAYMENT_Ratio'] = train['AMT_CREDIT'] / train['AMT_ANNUITY']
train['credit_to_goods_ratio'] = train['AMT_CREDIT'] / train['AMT_GOODS_PRICE']
train['DEBT_TO_INCOME'] = train['AMT_CREDIT'] / train['AMT_INCOME_TOTAL']
train['EMPLOYMENT'] = train['DAYS_EMPLOYED'] / train['DAYS_BIRTH']
train['INCOME_TO_CREDIT'] = train['AMT_INCOME_TOTAL'] / train['AMT_CREDIT']
train['AMT_CHILDREN'] = train['AMT_INCOME_TOTAL'] / (1 + train['CNT_CHILDREN'])
train['income_to_family'] = train['AMT_INCOME_TOTAL'] / train['CNT_FAM_MEMBERS']
train['Payment_credit'] = train['AMT_ANNUITY'] / train['AMT_CREDIT']
train['phone_to_birth_ratio'] = train['DAYS_LAST_PHONE_CHANGE'] / train['DAYS_BIRTH']
train['phone_to_employ_ratio'] = train['DAYS_LAST_PHONE_CHANGE'] / train['DAYS_EMPLOYED']
installments_payments["Difference"]=installments_payments["AMT_INSTALMENT"]-installments_payments["AMT_PAYMENT"]
previous_application["Per_for_DP"]=previous_application["AMT_DOWN_PAYMENT"]/previous_application["AMT_CREDIT"]

test['annuity_income_percentage'] = test['AMT_ANNUITY'] / test['AMT_INCOME_TOTAL']
test['car_to_birth_ratio'] = test['OWN_CAR_AGE'] / test['DAYS_BIRTH']
test['car_to_employ_ratio'] = test['OWN_CAR_AGE'] / test['DAYS_EMPLOYED']
test['children_ratio'] = test['CNT_CHILDREN'] / test['CNT_FAM_MEMBERS']
test['Annual_PAYMENT_Ratio'] = test['AMT_CREDIT'] / test['AMT_ANNUITY']
test['credit_to_goods_ratio'] = test['AMT_CREDIT'] / test['AMT_GOODS_PRICE']
test['DEBT_TO_INCOME'] = test['AMT_CREDIT'] / test['AMT_INCOME_TOTAL']
test['EMPLOYMENT'] = test['DAYS_EMPLOYED'] / test['DAYS_BIRTH']
test['INCOME_TO_CREDIT'] = test['AMT_INCOME_TOTAL'] / test['AMT_CREDIT']
test['AMT_CHILDREN'] = test['AMT_INCOME_TOTAL'] / (1 + test['CNT_CHILDREN'])
test['income_to_family'] = test['AMT_INCOME_TOTAL'] / test['CNT_FAM_MEMBERS']
test['Payment_credit'] = test['AMT_ANNUITY'] / test['AMT_CREDIT']
test['phone_to_birth_ratio'] = test['DAYS_LAST_PHONE_CHANGE'] / test['DAYS_BIRTH']
test['phone_to_employ_ratio'] = test['DAYS_LAST_PHONE_CHANGE'] / test['DAYS_EMPLOYED']

We remove these features/columns because in this most of the values were of 1 type and were not really contributing much in the identification of the TARGET variable

Selecting the categorical data from all the features

In [ ]:
columns=train.select_dtypes(exclude=["number","bool_"]).columns
print(columns)

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')


In [ ]:
for i in columns:
    if train[i].isnull().values.any():
        if((train[i].isnull().values.sum()/train.shape[0]*100)>60):
          train=train.drop(i,1)
          test=test.drop(i,1)
          continue
        else:
          train[i]=train[i].replace(np.nan,"XNA")
          test[i]=test[i].replace(np.nan,"XNA")

replacing all the categorical data with "XNA" type as its unidentified

In [ ]:
inde=train.columns
final=[]
final_test=[]
print("FOR TRAIN DATASET")
for i in inde:
    if i not in columns:
        if train[i].isnull().values.any():
            if((train[i].isnull().sum()/len(train.index)*100)>60):
              train=train.drop(i,1)
              test=test.drop(i,1)
              final_test.append(i)
              continue
            final.append(i)
            print(i+":\t\t"+str(train[i].isnull().sum()/len(train.index)*100)+" %")

FOR TRAIN DATASET
AMT_ANNUITY:		0.0039023501904021695 %
AMT_GOODS_PRICE:		0.09040444607765027 %
CNT_FAM_MEMBERS:		0.0006503916984003616 %
EXT_SOURCE_1:		56.38115555093055 %
EXT_SOURCE_2:		0.21462926047211933 %
EXT_SOURCE_3:		19.825564946489024 %
APARTMENTS_AVG:		50.750064226180214 %
BASEMENTAREA_AVG:		58.51639149677893 %
YEARS_BEGINEXPLUATATION_AVG:		48.78132855512232 %
ELEVATORS_AVG:		53.29634772541764 %
ENTRANCES_AVG:		50.34909774411639 %
FLOORSMAX_AVG:		49.76114364876247 %
LANDAREA_AVG:		59.376859713762606 %
LIVINGAREA_AVG:		50.19365412819871 %
NONLIVINGAREA_AVG:		55.17923169228668 %
APARTMENTS_MODE:		50.750064226180214 %
BASEMENTAREA_MODE:		58.51639149677893 %
YEARS_BEGINEXPLUATATION_MODE:		48.78132855512232 %
ELEVATORS_MODE:		53.29634772541764 %
ENTRANCES_MODE:		50.34909774411639 %
FLOORSMAX_MODE:		49.76114364876247 %
LANDAREA_MODE:		59.376859713762606 %
LIVINGAREA_MODE:		50.19365412819871 %
NONLIVINGAREA_MODE:		55.17923169228668 %
APARTMENTS_MEDI:		50.750064226180214 %
BASEMENTAR

In [ ]:
print(final_test)

['OWN_CAR_AGE', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'FLOORSMIN_AVG', 'LIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_AVG', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'FLOORSMIN_MODE', 'LIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_MODE', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'FLOORSMIN_MEDI', 'LIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'car_to_birth_ratio', 'car_to_employ_ratio']


In [ ]:
print(train.shape)

(307507, 109)


In [ ]:
from tqdm import tqdm
for i in tqdm(final):
    median=train[i].median()
    train[i].fillna(median, inplace=True)
    test[i].fillna(median,inplace=True)

100%|██████████| 53/53 [00:00<00:00, 202.71it/s]


Taking all the non-categorical values which have NAN in it. We take the median of that column and replace all the nan values with it

Finding top correlated features with the target variable

In [ ]:
correlation=train.corr()
print("TOP POSITIVE")
print(correlation["TARGET"].sort_values().head())
print("TOP NEGATIVE")
print(correlation["TARGET"].sort_values().tail())

TOP POSITIVE
EXT_SOURCE_2      -0.160294
EXT_SOURCE_3      -0.155899
EXT_SOURCE_1      -0.098887
DAYS_EMPLOYED     -0.044934
AMT_GOODS_PRICE   -0.039625
Name: TARGET, dtype: float64
TOP NEGATIVE
REGION_RATING_CLIENT           0.058901
REGION_RATING_CLIENT_W_CITY    0.060895
credit_to_goods_ratio          0.069405
DAYS_BIRTH                     0.078242
TARGET                         1.000000
Name: TARGET, dtype: float64


We take out the the top few correlated features from the train dataset with the target variable and then find its polynomial features.

In [ ]:

name=["EXT_SOURCE_2","EXT_SOURCE_3","REGION_RATING_CLIENT_W_CITY","DAYS_BIRTH","EXT_SOURCE_1"]
temp=train[name]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(5)
temp2=poly.fit_transform(temp)
n=[str(i) for i in range(len(temp2[0]))]
temp=pd.DataFrame(temp2,columns=n)
print(temp.head())

     0         1         2  ...           249          250       251
0  1.0  0.262949  0.139376  ...  5.124937e+04    -0.449804  0.000004
1  1.0  0.622246  0.535276  ...  8.476326e+06  -157.375678  0.002922
2  1.0  0.555912  0.729567  ...  4.699523e+07 -1248.529311  0.033170
3  1.0  0.650442  0.535276  ...  4.679312e+07 -1245.841623  0.033170
4  1.0  0.322738  0.535276  ...  5.146927e+07 -1306.609589  0.033170

[5 rows x 252 columns]


In [ ]:

temp["TARGET"]=train["TARGET"]
print(temp.shape)

(307507, 253)


In [ ]:
print(train.shape)
print(temp.shape)

(307507, 109)
(307507, 253)


In [ ]:
correlation=temp.corr()
print("TOP POSITIVE")
print(correlation["TARGET"].sort_values().head())
print("TOP NEGATIVE")
print(correlation["TARGET"].sort_values().tail())

TOP POSITIVE
29   -0.026137
7    -0.025811
64   -0.024981
22   -0.024328
10   -0.023747
Name: TARGET, dtype: float64
TOP NEGATIVE
149       0.023550
79        0.024239
28        0.024296
TARGET    1.000000
0              NaN
Name: TARGET, dtype: float64


In [ ]:
name_final=["29","7","64","22","28","79"]
temp2=temp[name_final]
temp=pd.DataFrame(temp2,columns=name_final)
print(temp.head())

         29         7        64        22           28           79
0  0.003043  0.036649  0.000800  0.009637  -346.733022   -28.791659
1  0.103675  0.333073  0.064511  0.207254 -5583.975307 -1738.108981
2  0.205220  0.405575  0.114084  0.225464 -7724.580288 -3908.621640
3  0.176171  0.348166  0.114589  0.226462 -6616.894625 -3348.134986
4  0.087413  0.172754  0.028212  0.055754 -3443.335521 -1742.320648


In [ ]:
print(train.shape)
print(temp.shape)
train=pd.concat([train,temp],axis=1)
train.dropna(subset = ["SK_ID_CURR"], inplace=True)
print(train.shape)

(307507, 109)
(307507, 6)
(307507, 115)


FOR TEST THE POLYNOMIAL FEATURES

In [ ]:
name=["EXT_SOURCE_2","EXT_SOURCE_3","REGION_RATING_CLIENT_W_CITY","DAYS_BIRTH","EXT_SOURCE_1"]
temp=test[name]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(5)
temp2=poly.fit_transform(temp)
n=[str(i) for i in range(len(temp2[0]))]
temp=pd.DataFrame(temp2,columns=n)
print(temp.head())

     0         1         2  ...           249          250       251
0  1.0  0.789654  0.159520  ...  1.578240e+08 -6173.307965  0.241470
1  1.0  0.291656  0.432962  ...  5.885056e+07 -1840.677016  0.057571
2  1.0  0.699787  0.610991  ...  5.201816e+07 -1313.558245  0.033170
3  1.0  0.509677  0.612704  ...  2.838337e+07 -1067.694673  0.040163
4  1.0  0.425687  0.535276  ...  1.404572e+06   -21.773565  0.000338

[5 rows x 252 columns]


In [ ]:
temp2=temp[name_final]
temp=pd.DataFrame(temp2,columns=name_final)
print(temp.head())

         29         7        64        22           28           79
0  0.094803  0.125965  0.074862  0.099469 -2423.698322 -1824.110478
1  0.071345  0.126276  0.020808  0.036829 -2281.043619 -1288.767295
2  0.216346  0.427564  0.151396  0.299203 -8567.521115 -4335.147954
3  0.164177  0.312281  0.083677  0.159163 -4364.443591 -2294.536289
4  0.046061  0.227860  0.019608  0.096997 -2971.298294  -600.633070


In [ ]:
print(test.shape)
test=pd.concat([test,temp],axis=1)
print(test.shape)

(48744, 108)
(48744, 114)


Changing all cateogircal features into their respective vector representation.
We use two types of encoding techniques.
- Response Coding is used when a particular feature has manu categories to avoid sparsity among the matrix.
- One hot encoding

In [ ]:
name_z=train.select_dtypes(exclude=["number"]).columns
print(name_z)

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'HOUSETYPE_MODE',
       'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')


In [ ]:
 def response(feature):
  name_z=train[feature].value_counts().index.tolist()
  zero=[]
  one=[]
  zero_t=[]
  one_t=[]
  for i in name_z:  
    total_count = train.loc[:,feature][(train[feature] == i)].count()
    p_0 = train.loc[:, feature][((train[feature] == i) & (train.TARGET==0))].count()
    p_1 = train.loc[:, feature][((train[feature] == i) & (train.TARGET==1))].count()
    zero.append(p_1/total_count)
    one.append(p_0/total_count)
  train[feature+"_p_0"]=train[feature]
  train[feature+"_p_1"]=train[feature]
  test[feature+"_p_0"]=test[feature]
  test[feature+"_p_1"]=test[feature]
  del train[feature]
  del test[feature]
  for index,i in enumerate(name_z):
    train[feature+"_p_0"].replace({i:zero[index]},inplace=True)
    train[feature+"_p_1"].replace({i:one[index]},inplace=True)
    test[feature+"_p_0"].replace({i:zero[index]},inplace=True)
    test[feature+"_p_1"].replace({i:one[index]},inplace=True)


  
    
def removing_extra(feature):
  temp_name=train[feature].value_counts().index.tolist()
  temp_values=[train[feature].value_counts()[index] for index,j in enumerate(temp_name)]
  for j in range(len(temp_name)):
    if (temp_values[j]/train.shape[0]*100<.3):
      train[i].replace({temp_name[j]:"XNA"},inplace=True)
      test[i].replace({temp_name[j]:"XNA"},inplace=True)

In [ ]:
for i in name_z:
  temp_name=train[i].value_counts().index.tolist()
  if(len(temp_name)>=20):
    response(i)
  else:
    removing_extra(i)

In [ ]:
name_z=train.select_dtypes(exclude=["number"]).columns
train=pd.get_dummies(train,columns=name_z)
print(train.shape)

(307507, 176)


In [ ]:
test=pd.get_dummies(test,columns=name_z)
print(test.shape)

(48744, 174)


We have got the top correlated features and its polynomial features added in both the test and train dataset

In [ ]:
for i in train.columns:
  if i not in test.columns and i!="TARGET":
    test[i]=[np.nan]*test.shape[0]

Checking for duplicate features and correlation in between features.
If any duplicate features found then we shall remove those as well as if features have a really high correlation then we shall remove those to decrease dimensionality

REFERENCE: https://towardsdatascience.com/2-types-of-duplicate-features-in-machine-learning-2931de6c949b

In [ ]:
pip install fast-ml

     |████████████████████████████████| 42 kB 486 kB/s 


In [ ]:
from fast_ml.feature_selection import get_duplicate_features
duplicate_features = get_duplicate_features(train)
duplicate_features.head(10)

,Desc,feature1,feature2
0,Duplicate Index,Annual_PAYMENT_Ratio,Payment_credit
1,Duplicate Index,DEBT_TO_INCOME,INCOME_TO_CREDIT
2,Duplicate Index,ORGANIZATION_TYPE_p_0,ORGANIZATION_TYPE_p_1
3,Duplicate Index,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans
4,Duplicate Index,CODE_GENDER_F,CODE_GENDER_M
5,Duplicate Index,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y
6,Duplicate Index,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y


In [ ]:
duplicate_features_list = duplicate_features.query("Desc=='Duplicate Values'")['feature2'].to_list()
train.drop(columns = duplicate_features_list, inplace=True)
test.drop(columns = duplicate_features_list, inplace=True)

In [ ]:
cori=train.corr()
print(cori.shape)

(176, 176)


In [ ]:
a=[]
b=[]
for i in range(cori.shape[0]):
  temp=[]
  for j in range(cori.shape[1]):
    if i!=j and abs(cori.iloc[[i],[j]].values)>.97:
      b.append(cori.index[i])
      b.append(cori.columns[j])
      if cori.index[i] not in a and cori.columns[j] not in a:
        a.append(cori.index[i])
        
final=[]
for i in set(b):
  if i not in a:
    final.append(i)
for i in final:
  del train[i]
  del test[i]

In [ ]:
print(train.shape)
print(test.shape)

(307507, 155)
(48744, 154)


# **BUREAU AND BUREAU_BALANCE**

In [ ]:
downloaded = drive.CreateFile({'id':'15gJEomRjwamYOCytXdmd18mU7h6G2f5p'}) # replace the id with id of file you want to access
downloaded.GetContentFile('bureau.csv') 


In [ ]:
downloaded = drive.CreateFile({'id':'14yzYIkGo63qu-UAdhygsN1tNAUIbAui9'}) # replace the id with id of file you want to access
downloaded.GetContentFile('bureau_balance.csv') 


In [ ]:
bureau = pd.read_csv('bureau.csv')
print(bureau.shape)
bureau_balance=pd.read_csv("bureau_balance.csv")
print(bureau_balance.shape)

(1716428, 17)
(27299925, 3)


Doing one hot encoding for the categorical feature in bureau balance file

In [ ]:
from sklearn.preprocessing import OneHotEncoder
temp=pd.get_dummies(bureau_balance.STATUS)
bureau_balance=bureau_balance.drop("STATUS",axis=1)
bureau_balance=bureau_balance.join(temp)
print(bureau_balance.head())

   SK_ID_BUREAU  MONTHS_BALANCE  0  1  2  3  4  5  C  X
0       5715448               0  0  0  0  0  0  0  1  0
1       5715448              -1  0  0  0  0  0  0  1  0
2       5715448              -2  0  0  0  0  0  0  1  0
3       5715448              -3  0  0  0  0  0  0  1  0
4       5715448              -4  0  0  0  0  0  0  1  0


In [ ]:
agge={"MONTHS_BALANCE":["min","max"],
      "0":["mean"],
      "1":["mean"],
      "2":["mean"],
      "3":["mean"],
      "4":["mean"],
      "5":["mean"],
      "C":['mean'],
      "X":["mean"]}

aggeragation for the feature months balance in bureau balance file

In [ ]:
bureau_balance_agg=bureau_balance.groupby("SK_ID_BUREAU").agg(agge)
new_name=[]
for i in bureau_balance_agg.columns:
  new_name.append("_".join(i))
new_name.append("SK_ID_BUREAU")
bureau_balance_agg["SK_ID_BUREAU"]=bureau_balance_agg.index
bureau_balance_agg.reset_index(drop=True, inplace=True)
bureau_balance_agg.head()

MONTHS_BALANCE             0    1  ...    5         C         X SK_ID_BUREAU
             min max      mean mean  ... mean      mean      mean             
0            -96   0  0.000000  0.0  ...  0.0  0.886598  0.113402      5001709
1            -82   0  0.060241  0.0  ...  0.0  0.578313  0.361446      5001710
2             -3   0  0.750000  0.0  ...  0.0  0.000000  0.250000      5001711
3            -18   0  0.526316  0.0  ...  0.0  0.473684  0.000000      5001712
4            -21   0  0.000000  0.0  ...  0.0  0.000000  1.000000      5001713

[5 rows x 11 columns]

In [ ]:
bureau_balance_agg=pd.DataFrame(bureau_balance_agg.values,columns=new_name)
print(bureau_balance_agg.head())

   MONTHS_BALANCE_min  MONTHS_BALANCE_max  ...    X_mean  SK_ID_BUREAU
0               -96.0                 0.0  ...  0.113402     5001709.0
1               -82.0                 0.0  ...  0.361446     5001710.0
2                -3.0                 0.0  ...  0.250000     5001711.0
3               -18.0                 0.0  ...  0.000000     5001712.0
4               -21.0                 0.0  ...  1.000000     5001713.0

[5 rows x 11 columns]


merging the bureau and bureau balance on sk_id_bureau

In [ ]:
print(bureau.shape)
bureau= pd.merge(bureau, bureau_balance_agg, how="left", on=["SK_ID_BUREAU"])
bureau=bureau.drop(["SK_ID_BUREAU"],axis=1)
print(bureau.shape)

(1716428, 17)
(1716428, 26)


getting the categorical features and numerical features for bureau.csv in seperate folders and performing the required operations such as one hot encoding, response coding and aggregation

In [ ]:
name=bureau.select_dtypes(exclude=["number"]).columns
rem_name={}

for i in bureau.columns:
  if i not in name and i not in new_name:
    rem_name[i]=[]

#del rem_name["SK_ID_CURR"]

In [ ]:
 def response(feature):
  name_z=bureau[feature].value_counts().index.tolist()
  name_value=bureau[feature].value_counts().tolist()
  zero=[]
  one=[]
  for i in name_z:  
    total_count = bureau.loc[:,feature][(bureau[feature] == i)].count()
    p_0 = bureau.loc[:, feature][((bureau[feature] == i) & (bureau.TARGET==0))].count()
    p_1 = bureau.loc[:, feature][((bureau[feature] == i) & (bureau.TARGET==1))].count()
    zero.append(p_1/total_count)
    one.append(p_0/total_count)
  bureau[feature+"_p_0"]=bureau[feature]
  bureau[feature+"_p_1"]=bureau[feature]
  del bureau[feature]
  for index,i in enumerate(name_z):
    bureau[feature+"_p_0"].replace({i:zero[index]},inplace=True)
    bureau[feature+"_p_1"].replace({i:one[index]},inplace=True)
def removing_extra(feature):
  temp_name=bureau[feature].value_counts().index.tolist()
  temp_values=[bureau[feature].value_counts()[index] for index,j in enumerate(temp_name)]
  for j in range(len(temp_name)):
    if (temp_values[j]/train.shape[0]*100<.3):
      bureau[i].replace({temp_name[j]:"XNA"},inplace=True)
      bureau[i].replace({temp_name[j]:"XNA"},inplace=True)

In [ ]:
bureau = pd.merge(bureau,train[['SK_ID_CURR','TARGET']],on='SK_ID_CURR', how='left')
print(bureau.shape)


(1716428, 27)


In [ ]:
for i in name:
  temp_name=bureau[i].value_counts().index.tolist()
  if(len(temp_name)>=10):
    response(i)
  else:
    removing_extra(i)
name=list(name)
name.remove("CREDIT_TYPE")

In [ ]:
bureau=pd.get_dummies(bureau,columns=name)
print(bureau.head())

   SK_ID_CURR  ...  CREDIT_CURRENCY_currency 2
0      215354  ...                           0
1      215354  ...                           0
2      215354  ...                           0
3      215354  ...                           0
4      215354  ...                           0

[5 rows x 33 columns]


In [ ]:
del bureau["TARGET"]
rem_name2=[]
for i in bureau.columns:
  if i not in rem_name and i not in new_name:
    rem_name2.append(i)


In [ ]:
new_name.remove("SK_ID_BUREAU")
del rem_name["SK_ID_CURR"]
for i in rem_name.keys():
  rem_name[i]=["min","max","mean"]
for i in rem_name2:
  rem_name[i]=["mean"]
for i in new_name:
  rem_name[i]=["mean"]


In [ ]:
bureau_agg=bureau.groupby("SK_ID_CURR").agg(rem_name)
new_name=[]
for i in bureau_agg.columns:
  new_name.append("_".join(i))
new_name.append("SK_ID_CURR")
bureau_agg["SK_ID_CURR"]=bureau_agg.index
bureau_agg.reset_index(drop=True, inplace=True)
bureau_agg.head()

DAYS_CREDIT                    ...    C_mean    X_mean SK_ID_CURR
          min  max         mean  ...      mean      mean           
0       -1572  -49  -735.000000  ...  0.441240  0.214590     100001
1       -1437 -103  -874.000000  ...  0.175426  0.161932     100002
2       -2586 -606 -1400.750000  ...       NaN       NaN     100003
3       -1326 -408  -867.000000  ...       NaN       NaN     100004
4        -373  -62  -190.666667  ...  0.128205  0.136752     100005

[5 rows x 56 columns]

In [ ]:
bureau_agg=pd.DataFrame(bureau_agg.values,columns=new_name)
print(bureau_agg.head())

   DAYS_CREDIT_min  DAYS_CREDIT_max  ...  X_mean_mean  SK_ID_CURR
0          -1572.0            -49.0  ...     0.214590    100001.0
1          -1437.0           -103.0  ...     0.161932    100002.0
2          -2586.0           -606.0  ...          NaN    100003.0
3          -1326.0           -408.0  ...          NaN    100004.0
4           -373.0            -62.0  ...     0.136752    100005.0

[5 rows x 56 columns]


Checking for dupicates

In [ ]:
from fast_ml.feature_selection import get_duplicate_features
duplicate_features = get_duplicate_features(bureau)
duplicate_features.head(10)

,Desc,feature1,feature2


Checking for correlation and removing those feaures with high correlation

In [ ]:
cori=bureau_agg.corr()
a=[]
b=[]
for i in range(cori.shape[0]):
  temp=[]
  for j in range(cori.shape[1]):
    if i!=j and abs(cori.iloc[[i],[j]].values)>.97:
      b.append(cori.index[i])
      b.append(cori.columns[j])
      if cori.index[i] not in a and cori.columns[j] not in a:
        a.append(cori.index[i])
print(a)
print(set(b)) 
final=[]
for i in set(b):
  if i not in a:
    final.append(i)
for i in final:
  del bureau_agg[i]

['DAYS_CREDIT_mean', 'CREDIT_ACTIVE_Active_mean']
{'CREDIT_ACTIVE_Active_mean', 'DAYS_CREDIT_mean', 'CREDIT_ACTIVE_Closed_mean', 'MONTHS_BALANCE_min_mean'}


Merging the file with test and train on basis of sk_id_curr

In [ ]:
print(train.shape)
train= pd.merge(train, bureau_agg, how="left", on=["SK_ID_CURR"])
print(train.shape)

(307507, 155)
(307507, 208)


In [ ]:
print(test.shape)
test= pd.merge(test, bureau_agg, how="left", on=["SK_ID_CURR"])
print(test.shape)

(48744, 154)
(48744, 207)


# **INSTALLMENT_PAYMENTS.CSV**

In [ ]:
downloaded = drive.CreateFile({'id':'1V3F5Pkoe0kxpzSibhAtBd9_TfjXxsRR_'})# replace the id with id of file you want to access
downloaded.GetContentFile('installments_payments.csv')

In [ ]:
installments_payments = pd.read_csv('installments_payments.csv')
print(installments_payments.shape)

(13605401, 8)


In [ ]:
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


We remove the sk_id_prev as its of no use

In [ ]:
installments_payments=installments_payments.drop("SK_ID_PREV",axis=1)
installments_payments.shape

(13605401, 7)

In [ ]:
installments_payments["Difference"]=installments_payments["AMT_INSTALMENT"]-installments_payments["AMT_PAYMENT"]

In [ ]:
names={}
for i in installments_payments.columns:
  names[i]=["min","max","mean"]
del names["SK_ID_CURR"]
print(names)

{'NUM_INSTALMENT_VERSION': ['min', 'max', 'mean'], 'NUM_INSTALMENT_NUMBER': ['min', 'max', 'mean'], 'DAYS_INSTALMENT': ['min', 'max', 'mean'], 'DAYS_ENTRY_PAYMENT': ['min', 'max', 'mean'], 'AMT_INSTALMENT': ['min', 'max', 'mean'], 'AMT_PAYMENT': ['min', 'max', 'mean'], 'Difference': ['min', 'max', 'mean']}


Getting the aggregation

In [ ]:
installments_payments=installments_payments.groupby("SK_ID_CURR").agg(names)
new_name=[]
for i in installments_payments.columns:
  new_name.append("_".join(i))
new_name.append("SK_ID_CURR")
installments_payments["SK_ID_CURR"]=installments_payments.index
installments_payments.reset_index(drop=True, inplace=True)
installments_payments.head()

NUM_INSTALMENT_VERSION                 ... Difference      SK_ID_CURR
                     min  max      mean  ...        max mean           
0                    1.0  2.0  1.142857  ...        0.0  0.0     100001
1                    1.0  2.0  1.052632  ...        0.0  0.0     100002
2                    1.0  2.0  1.040000  ...        0.0  0.0     100003
3                    1.0  2.0  1.333333  ...        0.0  0.0     100004
4                    1.0  2.0  1.111111  ...        0.0  0.0     100005

[5 rows x 22 columns]

In [ ]:
installments_payments=pd.DataFrame(installments_payments.values,columns=new_name)
print(installments_payments.head())

   NUM_INSTALMENT_VERSION_min  ...  SK_ID_CURR
0                         1.0  ...    100001.0
1                         1.0  ...    100002.0
2                         1.0  ...    100003.0
3                         1.0  ...    100004.0
4                         1.0  ...    100005.0

[5 rows x 22 columns]


Checking for duplicated and high correlation features

In [ ]:
from fast_ml.feature_selection import get_duplicate_features
duplicate_features = get_duplicate_features(installments_payments)
duplicate_features.head(10)

,Desc,feature1,feature2


In [ ]:
cori=installments_payments.corr()
a=[]
b=[]
for i in range(cori.shape[0]):
  temp=[]
  for j in range(cori.shape[1]):
    if i!=j and abs(cori.iloc[[i],[j]].values)>.97:
      b.append(cori.index[i])
      b.append(cori.columns[j])
      if cori.index[i] not in a and cori.columns[j] not in a:
        a.append(cori.index[i])
print(a)
print(set(b)) 
final=[]
for i in set(b):
  if i not in a:
    final.append(i)
for i in final:
  del installments_payments[i]

['DAYS_INSTALMENT_min', 'DAYS_INSTALMENT_max', 'DAYS_INSTALMENT_mean', 'AMT_INSTALMENT_max', 'AMT_INSTALMENT_mean']
{'DAYS_ENTRY_PAYMENT_max', 'AMT_INSTALMENT_mean', 'AMT_PAYMENT_mean', 'DAYS_INSTALMENT_min', 'AMT_INSTALMENT_max', 'DAYS_INSTALMENT_max', 'DAYS_ENTRY_PAYMENT_mean', 'AMT_PAYMENT_max', 'DAYS_ENTRY_PAYMENT_min', 'DAYS_INSTALMENT_mean'}


Merging the table with train and test on basis on SK_ID_CURR

In [ ]:
print(train.shape)
train= pd.merge(train, installments_payments, how="left", on=["SK_ID_CURR"])
print(train.shape)
print(test.shape)
test= pd.merge(test, installments_payments, how="left", on=["SK_ID_CURR"])
print(test.shape)

(307507, 208)
(307507, 224)
(48744, 207)
(48744, 223)


# **PREVIOUS_APPLICATION.CSV**

In [ ]:
downloaded = drive.CreateFile({'id':'1SkflEl6dAPwt6hRAlYs7C1_4z5j0tcuH'})# replace the id with id of file you want to access
downloaded.GetContentFile('previous_application.csv')

In [ ]:
previous_application = pd.read_csv('previous_application.csv')
print(previous_application.shape)

(1670214, 37)


In [ ]:
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


The features having days have entries which is equivalent to 1000 years which is not possible so removing those entries

In [ ]:
previous_application.DAYS_FIRST_DRAWING.replace(365243,np.nan,inplace=True)
previous_application.DAYS_FIRST_DUE.replace(365243,np.nan,inplace=True)
previous_application.DAYS_LAST_DUE_1ST_VERSION.replace(365243,np.nan,inplace=True)
previous_application.DAYS_LAST_DUE.replace(365243,np.nan,inplace=True)
previous_application.DAYS_TERMINATION.replace(365243,np.nan,inplace=True)
previous_application=previous_application.drop("SK_ID_PREV",axis=1)

In [ ]:
installments_payments["Difference"]=installments_payments["AMT_INSTALMENT"]-installments_payments["AMT_PAYMENT"]
previous_application["Per_for_DP"]=previous_application["AMT_DOWN_PAYMENT"]/previous_application["AMT_CREDIT"]

In [ ]:
columns=previous_application.select_dtypes(exclude=["number","bool_"]).columns
print(columns)

Index(['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype='object')


In [ ]:
rem_columns=[]
for i in previous_application.columns:
  if i not in columns:
    rem_columns.append(i)

Performing one hot ecnoding, Responde coding. For response coding we are adding the target column from the main dataset

In [ ]:
check=[]
from tqdm import tqdm
def response(feature):
  name_z=previous_application[feature].value_counts().index.tolist()
  name_value=previous_application[feature].value_counts().tolist()
  zero=[]
  one=[]
  for i in tqdm(name_z):  
    total_count = previous_application.loc[:,feature][(previous_application[feature] == i)].count()
    p_0 = previous_application.loc[:, feature][((previous_application[feature] == i) & (previous_application.TARGET==0))].count()
    p_1 = previous_application.loc[:, feature][((previous_application[feature] == i) & (previous_application.TARGET==1))].count()
    zero.append(p_1/total_count)
    one.append(p_0/total_count)
  previous_application[feature+"_p_0"]=previous_application[feature]
  previous_application[feature+"_p_1"]=previous_application[feature]
  check.append(feature+"_p_0")
  check.append(feature+"_p_1")
  del previous_application[feature]
  for index,i in tqdm(enumerate(name_z)):
    previous_application[feature+"_p_0"].replace({i:zero[index]},inplace=True)
    previous_application[feature+"_p_1"].replace({i:one[index]},inplace=True)
def removing_extra(feature):
  temp_name=previous_application[feature].value_counts().index.tolist()
  temp_values=[previous_application[feature].value_counts()[index] for index,j in enumerate(temp_name)]
  for j in range(len(temp_name)):
    if (temp_values[j]/train.shape[0]*100<.3):
      previous_application[i].replace({temp_name[j]:"XNA"},inplace=True)
      previous_application[i].replace({temp_name[j]:"XNA"},inplace=True)

In [ ]:
previous_application = pd.merge(previous_application,train[['SK_ID_CURR','TARGET']],on='SK_ID_CURR', how='left')

for i in columns:
  temp_name=previous_application[i].value_counts().index.tolist()
  if(len(temp_name)>=10):
    response(i)
  else:
    removing_extra(i)
rem_columns=list(rem_columns)


100%|██████████| 25/25 [00:05<00:00,  4.21it/s]
25it [00:05,  4.41it/s]
100%|██████████| 28/28 [00:06<00:00,  4.18it/s]
28it [00:06,  4.58it/s]
100%|██████████| 11/11 [00:02<00:00,  3.92it/s]
11it [00:02,  4.39it/s]
100%|██████████| 17/17 [00:04<00:00,  3.96it/s]
17it [00:03,  4.33it/s]


In [ ]:
del previous_application["TARGET"]

In [ ]:
print(columns)
columns=[]

for i in previous_application.columns:
  if i not in check and i not in rem_columns:
    columns.append(i)
print(columns)

Index(['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype='object')
['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NAME_YIELD_GROUP']


one hot encoding

In [ ]:
previous_application=pd.get_dummies(previous_application,columns=columns)

In [ ]:
columns=[]
for i in previous_application.columns:
  if i not in rem_columns:
    columns.append(i)

In [ ]:
names={}
for i in columns:
  names[i]=["mean"]
for i in check:
  names[i]=["mean"]
for i in rem_columns:
  names[i]=["min","max","mean"]
del names["SK_ID_CURR"]
print(names)

{'NAME_CASH_LOAN_PURPOSE_p_0': ['mean'], 'NAME_CASH_LOAN_PURPOSE_p_1': ['mean'], 'NAME_GOODS_CATEGORY_p_0': ['mean'], 'NAME_GOODS_CATEGORY_p_1': ['mean'], 'NAME_SELLER_INDUSTRY_p_0': ['mean'], 'NAME_SELLER_INDUSTRY_p_1': ['mean'], 'PRODUCT_COMBINATION_p_0': ['mean'], 'PRODUCT_COMBINATION_p_1': ['mean'], 'NAME_CONTRACT_TYPE_Cash loans': ['mean'], 'NAME_CONTRACT_TYPE_Consumer loans': ['mean'], 'NAME_CONTRACT_TYPE_Revolving loans': ['mean'], 'NAME_CONTRACT_TYPE_XNA': ['mean'], 'WEEKDAY_APPR_PROCESS_START_FRIDAY': ['mean'], 'WEEKDAY_APPR_PROCESS_START_MONDAY': ['mean'], 'WEEKDAY_APPR_PROCESS_START_SATURDAY': ['mean'], 'WEEKDAY_APPR_PROCESS_START_SUNDAY': ['mean'], 'WEEKDAY_APPR_PROCESS_START_THURSDAY': ['mean'], 'WEEKDAY_APPR_PROCESS_START_TUESDAY': ['mean'], 'WEEKDAY_APPR_PROCESS_START_WEDNESDAY': ['mean'], 'FLAG_LAST_APPL_PER_CONTRACT_N': ['mean'], 'FLAG_LAST_APPL_PER_CONTRACT_Y': ['mean'], 'NAME_CONTRACT_STATUS_Approved': ['mean'], 'NAME_CONTRACT_STATUS_Canceled': ['mean'], 'NAME_CONTRA

In [ ]:
previous_application=previous_application.groupby("SK_ID_CURR").agg(names)
new_name=[]
for i in previous_application.columns:
  new_name.append("_".join(i))
new_name.append("SK_ID_CURR")
previous_application["SK_ID_CURR"]=previous_application.index
previous_application.reset_index(drop=True, inplace=True)
previous_application.head()

NAME_CASH_LOAN_PURPOSE_p_0 NAME_CASH_LOAN_PURPOSE_p_1  ... Per_for_DP SK_ID_CURR
                        mean                       mean  ...       mean           
0                   0.070625                   0.782240  ...   0.105940     100001
1                   0.070625                   0.782240  ...   0.000000     100002
2                   0.071402                   0.776131  ...   0.050585     100003
3                   0.070625                   0.782240  ...   0.241719     100004
4                   0.071790                   0.773077  ...   0.111173     100005

[5 rows x 129 columns]

In [ ]:
previous_application=pd.DataFrame(previous_application.values,columns=new_name)
print(previous_application.head())

   NAME_CASH_LOAN_PURPOSE_p_0_mean  ...  SK_ID_CURR
0                         0.070625  ...    100001.0
1                         0.070625  ...    100002.0
2                         0.071402  ...    100003.0
3                         0.070625  ...    100004.0
4                         0.071790  ...    100005.0

[5 rows x 129 columns]


checking for duplicates and highly correlated features

In [ ]:
from fast_ml.feature_selection import get_duplicate_features
duplicate_features = get_duplicate_features(previous_application)
duplicate_features.head(10)

,Desc,feature1,feature2
0,Duplicate Values,NAME_CONTRACT_STATUS_Unused offer_mean,CODE_REJECT_REASON_CLIENT_mean
1,Duplicate Index,FLAG_LAST_APPL_PER_CONTRACT_N_mean,FLAG_LAST_APPL_PER_CONTRACT_Y_mean


In [ ]:
duplicate_features_list = duplicate_features.query("Desc=='Duplicate Values'")['feature2'].to_list()
previous_application.drop(columns = duplicate_features_list, inplace=True)


In [ ]:
cori=previous_application.corr()
a=[]
b=[]
for i in range(cori.shape[0]):
  temp=[]
  for j in range(cori.shape[1]):
    if i!=j and abs(cori.iloc[[i],[j]].values)>.97:
      b.append(cori.index[i])
      b.append(cori.columns[j])
      if cori.index[i] not in a and cori.columns[j] not in a:
        a.append(cori.index[i])
print(a)
print(set(b)) 
final=[]
for i in set(b):
  if i not in a:
    final.append(i)
for i in final:
  del previous_application[i]

['NAME_CONTRACT_TYPE_Consumer loans_mean', 'FLAG_LAST_APPL_PER_CONTRACT_N_mean', 'NAME_PAYMENT_TYPE_Cash through the bank_mean', 'AMT_APPLICATION_max', 'AMT_APPLICATION_mean', 'AMT_CREDIT_max', 'RATE_INTEREST_PRIMARY_min', 'RATE_INTEREST_PRIMARY_max', 'RATE_INTEREST_PRIVILEGED_min', 'RATE_INTEREST_PRIVILEGED_max', 'DAYS_DECISION_min', 'SELLERPLACE_AREA_min', 'DAYS_FIRST_DRAWING_min', 'DAYS_FIRST_DRAWING_max', 'DAYS_FIRST_DUE_min', 'DAYS_LAST_DUE_1ST_VERSION_min', 'DAYS_LAST_DUE_min', 'DAYS_LAST_DUE_mean']
{'AMT_GOODS_PRICE_max', 'AMT_APPLICATION_mean', 'SELLERPLACE_AREA_mean', 'DAYS_FIRST_DRAWING_max', 'DAYS_TERMINATION_min', 'RATE_INTEREST_PRIVILEGED_min', 'RATE_INTEREST_PRIMARY_mean', 'RATE_INTEREST_PRIVILEGED_mean', 'DAYS_LAST_DUE_mean', 'RATE_INTEREST_PRIMARY_min', 'AMT_CREDIT_max', 'FLAG_LAST_APPL_PER_CONTRACT_Y_mean', 'DAYS_TERMINATION_mean', 'DAYS_FIRST_DUE_min', 'DAYS_LAST_DUE_min', 'NAME_PAYMENT_TYPE_Cash through the bank_mean', 'DAYS_DECISION_min', 'DAYS_LAST_DUE_1ST_VERSION_

merging the file with train and test

In [ ]:
print(train.shape)
train= pd.merge(train, previous_application, how="left", on=["SK_ID_CURR"])
print(train.shape)

(307507, 224)
(307507, 340)


In [ ]:
print(test.shape)
test= pd.merge(test, previous_application, how="left", on=["SK_ID_CURR"])
print(test.shape)

(48744, 223)
(48744, 339)


In [ ]:
del installments_payments
del bureau

In [ ]:
del bureau_balance

In [ ]:
del previous_application

# **POS_CASH_balance.csv**

In [ ]:
downloaded = drive.CreateFile({'id':'1HcFNIi8o5eVtpztatoZd1Z6C_UuocH_b'})# replace the id with id of file you want to access
downloaded.GetContentFile('POS_CASH_balance.csv')

In [ ]:
pos_cash_balance = pd.read_csv('POS_CASH_balance.csv')
print(pos_cash_balance.shape)

(10001358, 8)


In [ ]:
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [ ]:
pos_cash_balance=pos_cash_balance.drop("SK_ID_PREV",axis=1)
pos_cash_balance.shape

(10001358, 7)

In [ ]:
columns=pos_cash_balance.columns
pos_cash_balance=pd.get_dummies(pos_cash_balance,columns=["NAME_CONTRACT_STATUS"])
pos_cash_balance.head()
rem=[]
rem2=[]
for i in columns:
  if i in pos_cash_balance:
    rem.append(i)
for i in pos_cash_balance.columns:
  if i not in rem:
    rem2.append(i)
print(rem)
print(rem2)

['SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF']
['NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Amortized debt', 'NAME_CONTRACT_STATUS_Approved', 'NAME_CONTRACT_STATUS_Canceled', 'NAME_CONTRACT_STATUS_Completed', 'NAME_CONTRACT_STATUS_Demand', 'NAME_CONTRACT_STATUS_Returned to the store', 'NAME_CONTRACT_STATUS_Signed', 'NAME_CONTRACT_STATUS_XNA']


In [ ]:
names={}
for i in rem:
  names[i]=["min","max","mean"]
for i in rem2:
  names[i]=["mean"]
del names["SK_ID_CURR"]
print(names)

{'MONTHS_BALANCE': ['min', 'max', 'mean'], 'CNT_INSTALMENT': ['min', 'max', 'mean'], 'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean'], 'SK_DPD': ['min', 'max', 'mean'], 'SK_DPD_DEF': ['min', 'max', 'mean'], 'NAME_CONTRACT_STATUS_Active': ['mean'], 'NAME_CONTRACT_STATUS_Amortized debt': ['mean'], 'NAME_CONTRACT_STATUS_Approved': ['mean'], 'NAME_CONTRACT_STATUS_Canceled': ['mean'], 'NAME_CONTRACT_STATUS_Completed': ['mean'], 'NAME_CONTRACT_STATUS_Demand': ['mean'], 'NAME_CONTRACT_STATUS_Returned to the store': ['mean'], 'NAME_CONTRACT_STATUS_Signed': ['mean'], 'NAME_CONTRACT_STATUS_XNA': ['mean']}


In [ ]:
pos_cash_balance=pos_cash_balance.groupby("SK_ID_CURR").agg(names)
new_name=[]
for i in pos_cash_balance.columns:
  new_name.append("_".join(i))
new_name.append("SK_ID_CURR")
pos_cash_balance["SK_ID_CURR"]=pos_cash_balance.index
pos_cash_balance.reset_index(drop=True, inplace=True)
pos_cash_balance.head()

MONTHS_BALANCE      ... NAME_CONTRACT_STATUS_XNA SK_ID_CURR
             min max  ...                     mean           
0            -96 -53  ...                      0.0     100001
1            -19  -1  ...                      0.0     100002
2            -77 -18  ...                      0.0     100003
3            -27 -24  ...                      0.0     100004
4            -25 -15  ...                      0.0     100005

[5 rows x 25 columns]

In [ ]:
pos_cash_balance=pd.DataFrame(pos_cash_balance.values,columns=new_name)
print(pos_cash_balance.head())

   MONTHS_BALANCE_min  ...  SK_ID_CURR
0               -96.0  ...    100001.0
1               -19.0  ...    100002.0
2               -77.0  ...    100003.0
3               -27.0  ...    100004.0
4               -25.0  ...    100005.0

[5 rows x 25 columns]


CHECKING FOR DUPLICATES AND HIGHLY CORRELATED FEATURES

In [ ]:
from fast_ml.feature_selection import get_duplicate_features
duplicate_features = get_duplicate_features(pos_cash_balance)
duplicate_features.head(10)

,Desc,feature1,feature2


In [ ]:
cori=pos_cash_balance.corr()
a=[]
b=[]
for i in range(cori.shape[0]):
  temp=[]
  for j in range(cori.shape[1]):
    if i!=j and abs(cori.iloc[[i],[j]].values)>.97:
      b.append(cori.index[i])
      b.append(cori.columns[j])
      if cori.index[i] not in a and cori.columns[j] not in a:
        a.append(cori.index[i])
print(a)
print(set(b)) 
final=[]
for i in set(b):
  if i not in a:
    final.append(i)
for i in final:
  del pos_cash_balance[i]

['CNT_INSTALMENT_max']
{'CNT_INSTALMENT_FUTURE_max', 'CNT_INSTALMENT_max'}


MERGING THE DATASETS

In [ ]:
print(train.shape)
train= pd.merge(train, pos_cash_balance, how="left", on=["SK_ID_CURR"])
print(train.shape)
print(test.shape)
test= pd.merge(test, pos_cash_balance, how="left", on=["SK_ID_CURR"])
print(test.shape)

(307507, 340)
(307507, 363)
(48744, 339)
(48744, 362)


In [ ]:
import gc
gc.collect()

129

In [ ]:
del pos_cash_balance

In [ ]:
gc.collect()

100

# **CREDIT CARD BALANCE.CSV**



In [ ]:
downloaded = drive.CreateFile({'id':'1jqyrLSpsVyOIR87ag4kAvRPOTNjmFeI7'})# replace the id with id of file you want to access
downloaded.GetContentFile('credit_card_balance.csv')

In [ ]:
credit_card_balance = pd.read_csv('credit_card_balance.csv')
print(credit_card_balance.shape)

(3840312, 23)


In [ ]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [ ]:
credit_card_balance=credit_card_balance.drop("SK_ID_PREV",axis=1)
credit_card_balance.shape

(3840312, 22)

In [ ]:
columns=credit_card_balance.columns
credit_card_balance=pd.get_dummies(credit_card_balance,columns=["NAME_CONTRACT_STATUS"])
credit_card_balance.head()
rem=[]
rem2=[]
for i in columns:
  if i in credit_card_balance:
    rem.append(i)
for i in credit_card_balance.columns:
  if i not in rem:
    rem2.append(i)
print(rem)
print(rem2)

['SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF']
['NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Approved', 'NAME_CONTRACT_STATUS_Completed', 'NAME_CONTRACT_STATUS_Demand', 'NAME_CONTRACT_STATUS_Refused', 'NAME_CONTRACT_STATUS_Sent proposal', 'NAME_CONTRACT_STATUS_Signed']


In [ ]:
names={}
for i in rem:
  names[i]=["min","max","mean"]
for i in rem2:
  names[i]=["mean"]
del names["SK_ID_CURR"]
print(names)

{'MONTHS_BALANCE': ['min', 'max', 'mean'], 'AMT_BALANCE': ['min', 'max', 'mean'], 'AMT_CREDIT_LIMIT_ACTUAL': ['min', 'max', 'mean'], 'AMT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'mean'], 'AMT_DRAWINGS_CURRENT': ['min', 'max', 'mean'], 'AMT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'mean'], 'AMT_DRAWINGS_POS_CURRENT': ['min', 'max', 'mean'], 'AMT_INST_MIN_REGULARITY': ['min', 'max', 'mean'], 'AMT_PAYMENT_CURRENT': ['min', 'max', 'mean'], 'AMT_PAYMENT_TOTAL_CURRENT': ['min', 'max', 'mean'], 'AMT_RECEIVABLE_PRINCIPAL': ['min', 'max', 'mean'], 'AMT_RECIVABLE': ['min', 'max', 'mean'], 'AMT_TOTAL_RECEIVABLE': ['min', 'max', 'mean'], 'CNT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'mean'], 'CNT_DRAWINGS_CURRENT': ['min', 'max', 'mean'], 'CNT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'mean'], 'CNT_DRAWINGS_POS_CURRENT': ['min', 'max', 'mean'], 'CNT_INSTALMENT_MATURE_CUM': ['min', 'max', 'mean'], 'SK_DPD': ['min', 'max', 'mean'], 'SK_DPD_DEF': ['min', 'max', 'mean'], 'NAME_CONTRACT_STATUS_Active': ['mean'

In [ ]:
credit_card_balance=credit_card_balance.groupby("SK_ID_CURR").agg(names)
new_name=[]
for i in credit_card_balance.columns:
  new_name.append("_".join(i))
new_name.append("SK_ID_CURR")
credit_card_balance["SK_ID_CURR"]=credit_card_balance.index
credit_card_balance.reset_index(drop=True, inplace=True)
credit_card_balance.head()

MONTHS_BALANCE      ... NAME_CONTRACT_STATUS_Signed SK_ID_CURR
             min max  ...                        mean           
0             -6  -1  ...                         0.0     100006
1            -75  -2  ...                         0.0     100011
2            -96  -1  ...                         0.0     100013
3            -18  -2  ...                         0.0     100021
4            -11  -4  ...                         0.0     100023

[5 rows x 68 columns]

In [ ]:

credit_card_balance=pd.DataFrame(credit_card_balance.values,columns=new_name)
print(credit_card_balance.head())

   MONTHS_BALANCE_min  ...  SK_ID_CURR
0                -6.0  ...    100006.0
1               -75.0  ...    100011.0
2               -96.0  ...    100013.0
3               -18.0  ...    100021.0
4               -11.0  ...    100023.0

[5 rows x 68 columns]


CHECKING FOR HIGHLY CORRELATED FEATURES AND DUPLICATES

In [ ]:
from fast_ml.feature_selection import get_duplicate_features
duplicate_features = get_duplicate_features(credit_card_balance)
duplicate_features.head(10)

,Desc,feature1,feature2
0,Duplicate Values,SK_DPD_min,SK_DPD_DEF_min
1,Duplicate Index,AMT_RECIVABLE_mean,AMT_TOTAL_RECEIVABLE_mean


In [ ]:
duplicate_features_list = duplicate_features.query("Desc=='Duplicate Values'")['feature2'].to_list()
credit_card_balance.drop(columns = duplicate_features_list, inplace=True)

In [ ]:
cori=credit_card_balance.corr()
a=[]
b=[]
for i in range(cori.shape[0]):
  temp=[]
  for j in range(cori.shape[1]):
    if i!=j and abs(cori.iloc[[i],[j]].values)>.97:
      b.append(cori.index[i])
      b.append(cori.columns[j])
      if cori.index[i] not in a and cori.columns[j] not in a:
        a.append(cori.index[i])
print(a)
print(set(b)) 
final=[]
for i in set(b):
  if i not in a:
    final.append(i)
for i in final:
  del credit_card_balance[i]

['MONTHS_BALANCE_min', 'AMT_BALANCE_min', 'AMT_BALANCE_max', 'AMT_BALANCE_mean', 'AMT_INST_MIN_REGULARITY_max', 'AMT_INST_MIN_REGULARITY_mean', 'AMT_PAYMENT_CURRENT_max', 'AMT_RECEIVABLE_PRINCIPAL_min', 'AMT_RECEIVABLE_PRINCIPAL_max', 'AMT_RECEIVABLE_PRINCIPAL_mean', 'AMT_RECIVABLE_min', 'AMT_RECIVABLE_max', 'AMT_RECIVABLE_mean', 'CNT_INSTALMENT_MATURE_CUM_max', 'SK_DPD_DEF_max']
{'AMT_BALANCE_min', 'AMT_PAYMENT_CURRENT_max', 'AMT_INST_MIN_REGULARITY_mean', 'CNT_INSTALMENT_MATURE_CUM_max', 'AMT_TOTAL_RECEIVABLE_mean', 'AMT_RECIVABLE_min', 'AMT_RECIVABLE_mean', 'AMT_PAYMENT_TOTAL_CURRENT_max', 'MONTHS_BALANCE_min', 'AMT_BALANCE_max', 'AMT_RECEIVABLE_PRINCIPAL_min', 'AMT_TOTAL_RECEIVABLE_min', 'SK_DPD_DEF_mean', 'AMT_INST_MIN_REGULARITY_max', 'AMT_TOTAL_RECEIVABLE_max', 'AMT_RECIVABLE_max', 'MONTHS_BALANCE_mean', 'AMT_BALANCE_mean', 'SK_DPD_DEF_max', 'AMT_RECEIVABLE_PRINCIPAL_max', 'CNT_INSTALMENT_MATURE_CUM_mean', 'AMT_RECEIVABLE_PRINCIPAL_mean'}


In [ ]:
print(train.shape)
train= pd.merge(train, credit_card_balance, how="left", on=["SK_ID_CURR"])
print(train.shape)
print(test.shape)
test= pd.merge(test, credit_card_balance, how="left", on=["SK_ID_CURR"])
print(test.shape)

(307507, 363)
(307507, 422)
(48744, 362)
(48744, 421)


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns             
from timeit import default_timer as timer
import os
import random
import csv
import json
import itertools
import pprint
import gc
import re

# import featuretools for automated feature engineering

#Import sklearn helper metrics and transformations
from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,classification_report,roc_curve,auc, f1_score

#Import models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb

#import library for hyperparameter optimization
from hyperopt import STATUS_OK
from hyperopt import hp, tpe, Trials, fmin

In [ ]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
train.to_csv('filetrain.csv')
test.to_csv("filetest.csv")